In [1]:
from datasets import load_dataset
from transformers import AutoFeatureExtractor  , AutoModelForImageClassification, TrainingArguments, Trainer, MobileNetV2ForImageClassification
from torch.utils.data import Dataset
from torchvision import transforms
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from PIL import Image
import torch
import numpy as np

/home/lundm/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-09 11:46:35.380178: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 11:46:35.691739: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741535195.787864   13504 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741535195.814155   13504 cuda_blas.cc:1418] Unable to r

In [8]:
#Load  dataset
dataset = load_dataset("Piro17/dataset-affecthqnet-fer2013")
#dataset = load_dataset("AutumnQiu/fer2013")

sample_train = 24000
sample_test = 3000

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset['test'] = dataset['train'].shuffle(seed=96).select(range(sample_test))
dataset['train'] = dataset['train'].shuffle(seed=23).select(range(sample_train))

test_valid_split = dataset['test'].train_test_split(test_size=0.65, seed=45)
dataset['test'] = test_valid_split['train']
dataset['validation'] = test_valid_split['test']

label2id = {
    "angry": 0,
    "disgust": 1,
    "fear": 2,
    "happy": 3,
    "sad": 4,
    "surprise": 5,
    "neutral": 6,
}

id2label = {v: k for k, v in label2id.items()}


#Import Model from HuggingFace
model = "google/mobilenet_v2_1.0_224"
#model = "microsoft/resnet-26"
#model = "microsoft/resnet-50"
feature_extractor = AutoFeatureExtractor.from_pretrained(model)
model = MobileNetV2ForImageClassification.from_pretrained(model)


/home/lundm/.local/lib/python3.10/site-packages/transformers/models/mobilenet_v2/feature_extraction_mobilenet_v2.py:28: FutureWarning: The class MobileNetV2FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use MobileNetV2ImageProcessor instead.
  warnings.warn(


In [ ]:
print(dataset['train']['label'][:5])

[5, 3, 5, 1, 2, 2, 6, 6, 0, 5, 2, 5, 3, 3, 3, 0, 0, 5, 2, 3, 0, 3, 5, 6, 6, 2, 4, 3, 6, 3, 2, 2, 2, 0, 3, 5, 2, 6, 4, 4, 2, 3, 1, 2, 0, 2, 0, 1, 0, 0, 3, 3, 4, 5, 1, 5, 5, 3, 3, 0, 6, 3, 6, 6, 2, 6, 0, 6, 5, 3, 2, 3, 5, 3, 0, 5, 0, 4, 6, 2, 6, 3, 5, 2, 3, 3, 2, 1, 4, 2, 3, 5, 5, 4, 2, 3, 1, 4, 6, 3, 3, 0, 0, 4, 3, 3, 3, 3, 0, 2, 1, 6, 3, 3, 0, 0, 3, 2, 0, 5, 3, 4, 6, 3, 0, 0, 3, 6, 2, 3, 5, 5, 3, 2, 3, 3, 1, 1, 3, 3, 2, 3, 0, 4, 3, 2, 0, 5, 4, 2, 2, 4, 6, 6, 6, 6, 3, 0, 0, 0, 4, 0, 0, 5, 2, 2, 4, 6, 3, 3, 5, 6, 2, 3, 3, 2, 6, 3, 2, 4, 4, 6, 2, 2, 0, 4, 6, 3, 5, 2, 0, 4, 2, 5, 4, 3, 4, 6, 6, 0, 2, 6, 3, 6, 2, 6, 6, 3, 0, 5, 6, 5, 1, 3, 5, 1, 6, 3, 6, 3, 3, 3, 6, 2, 5, 3, 2, 6, 2, 4, 4, 1, 5, 3, 3, 4, 4, 2, 5, 2, 0, 6, 5, 0, 5, 5, 4, 5, 0, 3, 5, 0, 3, 1, 4, 6, 3, 3, 3, 4, 5, 2, 6, 0, 0, 0, 5, 5, 4, 3, 4, 5, 0, 5, 3, 5, 3, 6, 3, 3, 0, 5, 6, 1, 2, 2, 6, 0, 4, 4, 0, 1, 0, 3, 3, 2, 3, 0, 5, 5, 3, 0, 4, 2, 1, 4, 6, 6, 1, 6, 5, 5, 3, 1, 4, 6, 0, 2, 3, 6, 3, 0, 3, 2, 5, 6, 3, 2, 3, 5, 0, 4, 2, 

In [49]:
# Define the transform function
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([img.convert("RGB") for img in example_batch['image']], return_tensors='pt')
    inputs['labels'] = example_batch['label']
    return inputs

# Apply the transform to the datasets
dataset['train'] = dataset['train'].map(transform, batched=True)
dataset['validation'] = dataset['validation'].map(transform, batched=True)

# Remove the 'image' column as it's now transformed
dataset['train'] = dataset['train'].remove_columns(['image'])
dataset['validation'] = dataset['validation'].remove_columns(['image'])

# Set the format for PyTorch
dataset.set_format(type='torch')

In [37]:
from transformers import Trainer

In [4]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # Convert numeric labels to string labels
    predicted_labels = [id2label[p] for p in predictions]
    true_labels = [id2label[l] for l in labels]

    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels, average='weighted')  # Use 'weighted' for multi-class

    return {"accuracy": accuracy, "f1": f1}

In [5]:
from transformers import EarlyStoppingCallback

In [50]:
#Training Args
training_args = TrainingArguments(
    output_dir='./huggingface_fer_model/results',
    num_train_epochs=25,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./huggingface_fer_model/logs',
    logging_steps=25,
    warmup_steps=5000,
    report_to=[],
    learning_rate=7e-5,
    weight_decay=0.055,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=2,
)

#Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [51]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyError: 784

In [29]:
eval_results = trainer.evaluate()
print(f"Validation Loss: {eval_results['eval_loss']:.4f}")
print(f"Validation Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"Validation Precision: {eval_results['eval_precision']:.4f}")
print(f"Validation Recall: {eval_results['eval_recall']:.4f}")
print(f"Validation F1 Score: {eval_results['eval_f1']:.4f}")

Validation Loss: 0.7770
Validation Accuracy: 0.7426
Validation Precision: 0.7466
Validation Recall: 0.7426
Validation F1 Score: 0.7435


In [32]:
model.save_pretrained('./resnet_50_affectnethq-fer2013_model')
feature_extractor.save_pretrained('./resnet_50 _affectnethq-fer2013_model')

['./resnet_50 _affectnethq-fer2013_model/preprocessor_config.json']

In [34]:
print(dataset['test']['label'][:5])

tensor([3, 1, 0, 2, 4])


In [3]:
label2id = {
    'Angry': 0,
    'Disgust': 1,
    'Fear': 2,
    'Happy': 3,
    'Neutral': 4,
    'Sad': 5,
    'Surprise': 6
}

Pruning

In [29]:
from transformers import pipeline

classifier = pipeline('feature-extraction', model='./mobilenet_v2_affectnethq-fer2013_model', feature_extractor='./mobilenet_v2_affectnethq-fer2013_model', device=0)

Device set to use cuda:0


In [30]:
predicted_labels = classifier([img.convert("RGB") for img in dataset['validation']['image'][:5]])
print(predicted_labels)

TypeError: 'NoneType' object is not callable

In [33]:
with torch.device("cuda"):
    model = AutoModelForImageClassification.from_pretrained('./mobilenet_v2_affectnethq-fer2013_model',
                                            torch_dtype=torch.float16,)
    feature_extractor = AutoFeatureExtractor.from_pretrained('./mobilenet_v2_affectnethq-fer2013_model', torch_dtype=torch.float16)

    classifier = pipeline('image-feature-extraction', model=model, feature_extractor=feature_extractor, device=0)

    predicted_labels = classifier(dataset['validation']['image'][:5])
    actual_labels = [list(label2id.keys())[list(label2id.values()).index(label)] for label in dataset['validation']['label'][:5]]
    print(predicted_labels)
    print(actual_labels)

Device set to use cuda:0


[[[0.12109375, 0.55810546875, 1.0224609375, 1.375, 0.262451171875, -0.0955810546875, -1.03515625, -0.595703125, 0.1998291015625, -1.2568359375, -0.541015625, 0.052947998046875, -0.6630859375, -0.76904296875, 0.57763671875, -1.2734375, -0.59619140625, -1.48046875, -1.330078125, -2.390625, -0.5234375, 0.335205078125, 0.88232421875, -1.2822265625, -0.85693359375, -0.568359375, -1.15234375, -0.61376953125, -0.36328125, -1.298828125, 0.53857421875, 0.2327880859375, -1.6455078125, -0.353515625, -0.2100830078125, -0.0309906005859375, 0.269775390625, 1.02734375, 0.0004744529724121094, -0.52490234375, 0.01788330078125, -1.4208984375, -0.412109375, -0.68505859375, -0.60400390625, -1.0068359375, -0.646484375, -0.56787109375, -0.783203125, -0.62060546875, -0.88037109375, -0.17333984375, -1.6279296875, 0.58154296875, -0.274169921875, 0.054473876953125, -1.001953125, 0.037811279296875, -0.619140625, 0.6650390625, -0.77294921875, -2.419921875, -0.86572265625, -0.0211944580078125, 1.0859375, -0.057373